# Random Forest

In [1]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint
from time import time
import pandas as pd
import os

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

In [3]:
merge_features = '"{}"'.format(os.path.join(current_folder, '..', 'Features', 'Merge features.ipynb'))
calcular_auc = '"{}"'.format(os.path.join(current_folder, '..', 'Calcular AUC.ipynb'))
set_de_entrenamiento_testing_y_prediccion = '"{}"'.format(os.path.join(
    current_folder,
    '..',
    'Set de entrenamiento, testing y predicción.ipynb'
))
hiperparametros_csv = os.path.join(current_folder, 'hiperparametros', 'random_forest.csv')

In [4]:
pd.options.mode.chained_assignment = None
%run $merge_features

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

In [5]:
assert(df_features.shape[0] == df['person'].unique().shape[0])

Cargo los sets de entrenamiento, testing y predicción.

In [6]:
%run $set_de_entrenamiento_testing_y_prediccion

## Entrenamiento rápido

In [7]:
training = labels_training.merge(df_features, how='inner', on='person')

Hiperparámetros:

In [8]:
param = {'n_estimators':100, 'max_depth':10}

In [9]:
regr = RandomForestRegressor(**param)
regr.fit(training.drop('label', axis=1), training['label'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

Predigo:

In [10]:
labels_test = labels_test.merge(df_features, how='inner', on='person')

In [11]:
labels_test['label_predicted'] = regr.predict(labels_test.drop('label', axis=1))

In [12]:
%run $calcular_auc

auc score: 0.889051084051084


## Feature importance

In [13]:
feature_importance = pd.DataFrame(data={
    'columna':training.drop('label', axis=1).columns,
    'importancia':regr.feature_importances_
}).set_index('columna')
feature_importance.sort_values('importancia', ascending=False)

,importancia
columna,
dias ultimo checkout,0.232297
screen_resolution_width std,0.060372
dias ultima compra,0.033849
days until 31-05 mean,0.026336
viewed product,0.023903
feature_hashing_timestamp_days_28,0.021554
screen_resolution_height mean,0.019266
screen_resolution_height std,0.019100
event_count,0.018363


# Hiperparámetros

En esta sección vamos a buscar los hiperparámetros de random forest con un Random Search y cross validation. Para construir este Random Search se usó como base el código de sklearn https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html#sphx-glr-auto-examples-model-selection-plot-randomized-search-py.

Hiperparámetros a probar. 

In [14]:
labels_with_features = labels.merge(df_features, how='inner', on='person')
regr = RandomForestRegressor()

In [15]:
param_dist = {
    "n_estimators": randint(1, 1000),
    "max_depth": [3, 9, 12, 15, None],
    "max_features": randint(1, labels_with_features.shape[1]),
    "min_samples_split": randint(2, 11),
    "min_samples_leaf": randint(2, 100),
    "bootstrap": [True, False]
}

splits = 10 # cantidad de splits en el cross validation
n_iter_search = 20 # cantidad de combinaciones, en total splits*n_iter_search RF a probar

Nota: hay más info en la consola desde la cual se corre jupyter.

Se puede aumentar *n_jobs* para que corra más procesos en paralelo, pero se corre el riesgo de que se cuelgue por falta de memoria. Recomiendo que prueben ir aumentando *n_jobs* con un *n_iter_search* bajo hasta encontrar el mayor *n_jobs* que se banque su compu.

In [16]:
random_search = RandomizedSearchCV(regr, param_distributions=param_dist, iid=False, refit=True, verbose=10,
                                   return_train_score=True, n_iter=n_iter_search, cv=splits,
                                   scoring=make_scorer(roc_auc_score), n_jobs=2);

start = time()
random_search.fit(labels_with_features.drop('label', axis=1), labels_with_features['label'])
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:  3.2min
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:  6.0min
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed: 14.0min
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed: 14.2min
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed: 17.0min
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed: 23.9min
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed: 30.8min
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 32.5min
[Parallel(n_jobs=2)]: Done  57 tasks      | elapsed: 39.4min
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed: 41.2min
[Parallel(n_jobs=2)]: Done  81 tasks      | elapsed: 48.9min
[Parallel(n_jobs=2)]: Done  94 tasks      | elapsed: 56.8min
[Parallel(n_jobs=2)]: Done 109 tasks      | elapsed: 64.9min
[Parallel(n_jobs=2)]: Done 124 tasks      | elapsed: 69.1min
[Parallel(n_jobs=2)]: Done 141 tasks      | elapsed: 76.1min
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed: 96.6min
[Parallel(n_jobs=2)]: Do

RandomizedSearchCV took 7631.55 seconds for 20 candidates parameter settings.


El **mejor** Random Forest fue:

In [19]:
print('score: {}'.format(random_search.best_score_))
random_search.best_params_

score: 0.8771930215994177


{'bootstrap': True,
 'max_depth': 9,
 'max_features': 37,
 'min_samples_leaf': 30,
 'min_samples_split': 7,
 'n_estimators': 784}

El resultado de la búsqueda la podemos importar a un DataFrame de Pandas y analizarlo.

In [20]:
stats_training = pd.DataFrame(data=random_search.cv_results_)
stats_training.head(2)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bootstrap,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,...,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
0,159.415562,4.593273,0.184806,0.037099,True,12,81,15,7,636,"{'bootstrap': True, 'max_depth': 12, 'max_feat...",0.866330,...,0.951610,0.951909,0.952092,0.951903,0.951359,0.953499,0.950522,0.952397,0.952049,0.951177,0.951852,0.000751
1,7.942321,0.348438,0.024235,0.004209,False,None,29,87,2,59,"{'bootstrap': False, 'max_depth': None, 'max_f...",0.863274,...,0.919832,0.918047,0.918842,0.919527,0.918467,0.919935,0.917920,0.918561,0.918789,0.918826,0.918875,0.000659


Escribo el mejor resultado en un archivo.

In [21]:
data = random_search.best_params_.copy()
data['features'] = ','.join([f for f in labels_with_features.columns if f != 'label'])
data['auc'] = random_search.best_score_
data['cv'] = splits

In [25]:
mejores_resultados = pd.read_csv(hiperparametros_csv, index_col='fecha')
mejores_resultados.head(10)

,bootstrap,max_depth,max_features,min_samples_leaf,min_samples_split,n_estimators,cv,auc,features
fecha,,,,,,,,,
2018-11-24 19:50,False,9.0,65,91,7,104,10,0.859541,"screen_resolution_height,screen_resolution_wid..."
2018-11-24 20:51,False,NaN,55,67,10,253,10,0.860532,"screen_resolution_height,screen_resolution_wid..."
2018-11-26 17:13,False,15.0,97,83,7,994,10,0.860743,"screen_resolution_height,screen_resolution_wid..."
2018-11-27 00:28,True,NaN,86,77,8,186,10,0.864988,"screen_resolution_height mean,screen_resolutio..."
2018-11-29 18:11,True,9.0,37,30,7,784,10,0.877193,"screen_resolution_height mean,screen_resolutio..."


In [23]:
mejor_resultado = pd.DataFrame(data=data, index=[pd.datetime.now().strftime("%Y-%m-%d %H:%M")])
mejor_resultado.index.name = 'fecha'
mejores_resultados = mejores_resultados.append(mejor_resultado, sort=False)

In [24]:
mejores_resultados.to_csv(hiperparametros_csv)